In [5]:
!pip install pdfminer.six



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Seif\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from pdfminer.high_level import extract_text
text = extract_text('seif.pdf')

In [8]:
extracted_text={}

In [14]:
import re
import spacy

ModuleNotFoundError: No module named 'spacy'

*Extracting E-mail ID*

In [ ]:
match=re.search(r'[\w\.-]+@[\w\.-]+',text)
match.group(0)#E-MAIL
import re
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

email = get_email_addresses(text)
print(email)
extracted_text['E-mail'] = email

*Extracting Phone Number*

In [ ]:
def get_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

phone_number= get_phone_numbers(text)
print(phone_number)
extracted_text['Phone number'] = phone_number

*Extracting Name*


In [ ]:
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
  nlp_text = nlp(resume_text)
  # First name and Last name are always Proper Nouns
  pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
  
  matcher.add('NAME', [pattern], on_match=None)
  
  matches = matcher(nlp_text)
  
  for match_id, start, end in matches:
    span = nlp_text[start:end]
    return span.text

name = extract_name(text)
print(name)
extracted_text['Name'] = name    


*Extracting Skills*


In [ ]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

def extract_skills(resume_text):
  nlp_text = nlp(resume_text)
  
  # removing stop words and implementing word tokenization
  tokens = [token.text for token in nlp_text if not token.is_stop]

  skills = ["HTML", "CSS", "PHP", "MongoDB", "ReactJS", "SQL", "MSQL", "Github", "Springboot", "AngularJS", "ai", "Django", "Pyhton"]

  skillset = []
  
  # check for one-grams (example: python)
  for token in tokens:
    if token.lower() in skills:
      skillset.append(token)

  # check for bi-grams and tri-grams (example: machine learning)
  for token in nlp_text.noun_chunks:
    token = token.text.lower().strip()
    if token in skills:
      skillset.append(token)  
  return [i.capitalize() for i in set([i.lower() for i in skillset])]

extracted_text['Skills'] = extract_skills(text)




*Extracting Degree*

In [ ]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
Degree = [
            'BE','B.E.', 'B.E', 'BS', 'B.S','C.A.','c.a.','B.Com','B. Com',
            'M. Com', 'M.Com','M. Com .',
            'ME', 'M.E', 'M.E.', 'MS', 'M.S',
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH',
            'PHD', 'phd', 'ph.d', 'Ph.D.','MBA','mba',
            'graduate', 'post-graduate','5 year integrated masters','masters',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII', 'engineer'
        ]

def extract_degree(resume_text):
  nlp_text = nlp(resume_text)
  
  # Sentence Tokenizer
  nlp_text = [sent.text.strip() for sent in nlp_text.sents]
  
  edu = {}
  # Extract education degree
  for index, text in enumerate(nlp_text):
    for tex in text.split():
      # Replace all special symbols
      tex = re.sub(r'[?|$|.|!|,]', r'', tex)
      if tex.upper() in Degree and tex not in STOPWORDS:
        edu[tex] = text + nlp_text[index + 1]

  # Extract year
  education = []
  for key in edu.keys():
    year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
    if year:
      education.append((key, ''.join(year[0])))
    else:
      education.append(key)
  return education

extract_degree(text)
extracted_text['Degree'] = extract_degree(text)




*Extracting University/College*

In [ ]:
sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* University','[A-Z][a-z]* Educational Institute','[A-Z][a-z]* University',
                'University of [A-Z][a-z]* [A-Z][a-z]*', 'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]* [A-Z][a-z]*']
pattern = '({})'.format('|'.join(sub_patterns))
education = re.findall(pattern, text)
extracted_text['Education'] = education

*Extracting Experience*

In [ ]:
sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.','[A-Z][a-z]* [A-Z][a-z]* Private Limited', '[A-Z][a-z]* LLC',
                '[A-Z][a-z]* Pvt. Ltd.', '[A-Z][a-z]* Private Limited', '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.']
pattern = '({})'.format('|'.join(sub_patterns))
experience = re.findall(pattern, text)
extracted_text['Experience'] = experience

In [ ]:
print(extracted_text)


In [ ]:
import json
# Serializing json 
json_object = json.dumps(extracted_text, indent =8)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)
    

print(json_object)